Social Media Analytics Project 8 - Community Detection in a Twitter Network
===
Goloviatinski Sergiy, Herbelin Ludovic <br />
MCS 2020

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import networkx as nx
from math import sqrt, log
from tqdm.notebook import trange, tqdm
import random

## Dataset loading

In [2]:
DATA_COMBINED_PATH = 'data/twitter_combined.txt'
DATA_OTHERS = 'data/twitter/'

In [3]:
original_G = nx.Graph()

edges = nx.read_edgelist(DATA_COMBINED_PATH)

original_G.add_edges_from(edges.edges())

print(f"Number of nodes : {len(original_G.nodes)}")
print(f"Number of edges : {len(original_G.edges())}")

Number of nodes : 81306
Number of edges : 1342310


### Reducing the size of the graph

In [4]:
def random_walk(G,n):
    
    node=random.choice(list(G.nodes))
    
    visited = set()
    visited.add(node)
    
    while len(visited)<n:
        node=random.choice(list(G.neighbors(node)))
        visited.add(node)
    
    visited=list(visited)
    
    # we copy the graph because some attributes are shared with the original graph after calling subgraph method
    G=G.copy()
    return G.subgraph(visited)

In [5]:
# Reduce the graph size, with random walk
N_NODES = 10

G = random_walk(original_G,N_NODES)

print(f"Number of nodes : {len(G.nodes)}")
print(f"Number of edges : {len(G.edges())}")

Number of nodes : 10
Number of edges : 20


## Implementation

### Girvan-Newman

In [6]:
def compute_gn(G, n_iter):
    G=G.copy()
    n_nodes=len(G.nodes)
    nodes_affected=set()
    
    for i in range(n_iter):
        betweennesses=nx.edge_betweenness_centrality(G,normalized=False)
        max_betweenness=max(betweennesses.values())
        for edge, betweeness in betweennesses.items():
            # to remove n edges that all have the same value which is equals to max
            if betweeness == max_betweenness:
                G.remove_edge(edge[0],edge[1])
                nodes_affected.add(edge[0])
                nodes_affected.add(edge[1])
        
    # hypothesis: each node affected by the edge removal will be the starting point for a community.
    # create a dict with community id as key, and set of nodes in this community as value
    communities=dict(zip(list(range(len(nodes_affected))),[set([node]) for node in nodes_affected]))
    
    duplicate_communities=True
    
    while duplicate_communities:
        n_nodes_in_communities=0
        # while all the original nodes are not affected to a community
        while n_nodes_in_communities<n_nodes:
            for _, community in communities.items():
                # Random walk to add nodes to communities
                node=random.choice(list(community))
                try:
                    community.add(random.choice(list(G.neighbors(node))))
                except IndexError:
                    # If a node has no neighbor, it's a community on its own
                    pass

            n_nodes_in_communities=len(set().union(*list(communities.values())))

            
        # Idea to elimite duplicate communities: a duplicate community is created if two starting nodes
        # (in nodes_affected list) are reachable from one to another, to avoid duplicating communities, we restart
        # the whole process after removing one of the two starting nodes which are in the same community
        # from the nodes_affected list
        duplicate_communities=False
        for i in communities.keys():
            for j in communities.keys():
                # iterate through the "matrix" in a triangular way: avoid to compare a community with itself
                # and avoid to compare B with A after we already compared A with B
                if i>j:
                    nodes_affected_size=len(nodes_affected)
                    for node in communities[i]:
                        if node in communities[j]:
                            duplicate_communities=True
                            # The goal is to remove a node that appears in 2 communities from the starting nodes
                            if len(nodes_affected)<nodes_affected_size:
                                break
                            try:
                                nodes_affected.remove(node)
                            except KeyError:
                                pass
        
        if duplicate_communities:
            communities=dict(zip(list(range(len(nodes_affected))),[set([node]) for node in nodes_affected]))
    
    return communities

### Cosine similarity

In [7]:
def cosine_sim(vi_neighbors, vj_neighbors):
    return len(vi_neighbors.intersection(vj_neighbors)) / sqrt(len(vi_neighbors) * len(vj_neighbors))

def compute_cosine_sim(G, selected_nodes):
    nodes_similarities = {}
    
    # TODO : optimize not to compute multiple times the same product maybe triangular matrix
    for node in selected_nodes:
        vi_neighbors = set(G[node])
        for neighbor in vi_neighbors:
            vj_neighbors = set(G[neighbor])
            sim = cosine_sim(vi_neighbors, vj_neighbors)
            nodes_similarities[(node, neighbor)] = sim
    
    return nodes_similarities

### Adamic-Adar similarity

In [8]:
def adamic_adar_sim(G, vi_neighbors, vj_neighbors):
    common_neighbors = vi_neighbors.intersection(vj_neighbors)
    
    # sum of 1 / log(nb of neighbors for each common neighbor to vi and vj)
    return sum([1 / log(len(G[neighbor])) for neighbor in common_neighbors])
        

def compute_adamic_adar_sim(G, selected_nodes):
    nodes_similarities = {}
    
    # TODO : optimize not to compute multiple times the same product maybe triangular matrix
    for node in selected_nodes:
        vi_neighbors = set(G[node])
        for neighbor in vi_neighbors:
            vj_neighbors = set(G[neighbor])
            sim = adamic_adar_sim(G, vi_neighbors, vj_neighbors)
            nodes_similarities[(node, neighbor)] = sim
    
    return nodes_similarities

## Analysis

### Compute the clusters and evaluate with different values of iteration level

In [9]:
iterations=2
compute_gn(G,iterations)

{0: {'59836995'},
 1: {'25276905', '806283'},
 2: {'16870573'},
 3: {'130574973', '16273438', '18393753', '23133578', '25147689', '62686586'}}

In [10]:
# to compare with method from networkx
from networkx.algorithms.community.centrality import girvan_newman

comp = girvan_newman(G)

for i in range(iterations-1):
    next(comp)
for com in next(comp):
    print(com)

{'59836995'}
{'62686586', '16273438', '18393753', '130574973', '25147689', '23133578'}
{'16870573', '25276905', '806283'}


### Find the top K users

In [24]:
k = 10

if k > len(G):
    print(f"Warning : K chosen : {k} is higher than the number of nodes in the graph : {len(G)}\n")

nodes_degrees = dict(G.degree())
# normalize the node degrees using the max node degree
max_deg = max(nodes_degrees.values())
nodes_degrees = {node:deg /float(max_deg) for node, deg in nodes_degrees.items()}

    
# sort the node:degree dictionary
nodes_degrees= {node: deg for node, deg in sorted(nodes_degrees.items(), key=lambda item: item[1], reverse=True)}


top_nodes = list(nodes_degrees)[:k]
top_nodes_dict = {node:nodes_degrees[node] for node in top_nodes}
print(f"{k} nodes with highest degree : {top_nodes_dict}")

{'59836995': 0.6, '62686586': 1.0, '25276905': 0.6, '16273438': 0.8, '18393753': 1.0, '130574973': 1.0, '16870573': 0.4, '25147689': 1.0, '806283': 0.6, '23133578': 1.0}
10 nodes with highest degree : {'62686586': 1.0, '18393753': 1.0, '130574973': 1.0, '25147689': 1.0, '23133578': 1.0, '16273438': 0.8, '59836995': 0.6, '25276905': 0.6, '806283': 0.6, '16870573': 0.4}


### Find the most similar nodes

In [12]:
similarities_tested = {
    'cosine':compute_cosine_sim,
    'adamic-adar':compute_adamic_adar_sim,
}


for similarity_label, similarity_func in similarities_tested.items():
    top_nodes_sims = similarity_func(G, top_nodes)
    most_similar_pair = max(top_nodes_sims, key=top_nodes_sims.get)
    
    print(f"Most similar nodes using function : {similarity_label} are {most_similar_pair} with similarity value : {top_nodes_sims[most_similar_pair]:.3f}") 

ValueError: max() arg is an empty sequence

## Visualization